# Setting Envrionments and Aquiring Data

In [ ]:
!nvidia-smi
!pip install roboflow
!pip install ultralytics
from ultralytics import YOLO
from roboflow import Roboflow
import os, random
from IPython.display import display, Image
import ultralytics
import glob
from moviepy.editor import *
import yaml

In [ ]:
# Loading Dataset A

# Security breach here, but for your convineicence I will leave my API key open.
# Will be closed after the grading!
rf = Roboflow(api_key="Ek2vKP2jbMY6qenMh678")
project = rf.workspace("visionscorer").project("bigger-dataset")
version = project.version(2)
dataset = version.download("yolov11")

In [ ]:
# Loading Dataset B

# Security breach here, but for your convineicence I will leave my API key open.
# Will be closed after the grading!
rf = Roboflow(api_key="Ek2vKP2jbMY6qenMh678")
project = rf.workspace("visionscorer").project("basketball-players-fy4c2-rzrjr")
version = project.version(1)
dataset = version.download("yolov11")

In [ ]:
'''

# Uncomment this section, if you want to try finetuning your
# own model with different settings, such as different epochs and imgsz

# If you decide to do so, don't forget to change directory of the models on the coding blocks below.


# Using smaller dataset ('Basketball-Players-1') with setting below took 5 minutes to run on Google Colab.
!yolo task=detect mode=train model=yolo11n.pt data=/content/Basketball-Players-1/data.yaml epochs=100 imgsz=640

# Using bigger dataset ('Bigger-Dataset-2') with setting below took 20 minutes to run on Google Colab.
!yolo task=detect mode=train model=yolo11n.pt data=/content/Bigger-Dataset-2/data.yaml epochs=20 imgsz=640

'''

In [ ]:
# Obtaining the fine-tuned model from project github.

# b_best: Epochs = 20, Image Size = 640
!wget https://raw.githubusercontent.com/Br19ht/VisionScorer/main/b_best.pt
# s_best: Epochs = 100, Image Size = 640
!wget https://raw.githubusercontent.com/Br19ht/VisionScorer/main/s_best.pt

# Model Validation

In [ ]:
# Validation results with validation set, for b_best.pt model with Dataset A
!yolo task=detect mode=val model=/content/b_best.pt data=/content/Bigger-Dataset-2/data.yaml

In [ ]:
# Validation results with validation set, for s_best.pt model with Dataset B
!yolo task=detect mode=val model=/content/s_best.pt data=/content/Basketball-Players-1/data.yaml

# Image Experiments

In [ ]:
# Object Detection on the testing split of the Dataset A. Contains 727 images.

!yolo task=detect mode=predict model=/content/b_best.pt conf=0.5 source = /content/Bigger-Dataset-2/test/images data=/Bigger-Dataset-2/data.yaml

In [9]:
# Commands to change validation file path to be test set file path,
# so that the model can validate the testing set results.

with open("/content/Bigger-Dataset-2/data.yaml") as f:
     list_doc = yaml.safe_load(f)

list_doc["val"] = "../test/images"

with open("/content/Bigger-Dataset-2/test-to-val-data.yaml", "w") as f:
    yaml.dump(list_doc, f)

In [ ]:
# Validation results with test set, for b_best.pt model with Dataset A

!yolo task=detect mode=val model=/content/b_best.pt data=/content/Bigger-Dataset-2/test-to-val-data.yaml

In [ ]:
display(Image(filename='/content/runs/detect/val3/F1_curve.png'))

In [ ]:
display(Image(filename='/content/runs/detect/val3/PR_curve.png'))

In [ ]:
display(Image(filename='/content/runs/detect/val3/confusion_matrix.png'))

In [ ]:
# Object Detection on the testing split of the Dataset B. Contains 24 images.

!yolo task=detect mode=predict model=/content/s_best.pt conf=0.5 source = /content/Basketball-Players-1/test/images data=/Basketball-Players-1/data.yaml

In [15]:
# Commands to change validation file path to be test set file path,
# so that the model can validate the testing set results.

with open("/content/Basketball-Players-1/data.yaml") as f:
     list_doc = yaml.safe_load(f)

list_doc["val"] = "../test/images"

with open("/content/Basketball-Players-1/test-to-val-data.yaml", "w") as f:
    yaml.dump(list_doc, f)

In [ ]:
# Validation results with test set, for s_best.pt model with Dataset B

!yolo task=detect mode=val model=/content/s_best.pt data=/content/Basketball-Players-1/test-to-val-data.yaml

In [ ]:
display(Image(filename='/content/runs/detect/val4/F1_curve.png'))

In [ ]:
display(Image(filename='/content/runs/detect/val4/PR_curve.png'))

In [ ]:
display(Image(filename='/content/runs/detect/val4/confusion_matrix.png'))

# Image Experiment Results

In [ ]:
# Displaying Random Pictures generated from the test set object detection tasks,
# for b_best.pt model with Dataset A.

# If you want to see more results, click the play button and see different images!

for i in range(10):
    img_path = os.path.join('/content/runs/detect/predict/', random.choice(os.listdir('/content/runs/detect/predict/')))
    display(Image(filename=img_path))
    print("\n")

In [ ]:
# Displaying Random Pictures generated from the test set object detection tasks,
# for s_best.pt model with Dataset B.

# If you want to see more results, click the play button and see different images!

for i in range(10):
    img_path = os.path.join('/content/runs/detect/predict2/', random.choice(os.listdir('/content/runs/detect/predict2/')))
    display(Image(filename=img_path))
    print("\n")

# Video Experiment

In [ ]:
# Obtaining the 28 seconds video from project github.
# Source: https://www.nba.com/lakers/videos/lebron-luka-austin-combine-for-85-points-and-defeat-memphis

!wget https://raw.githubusercontent.com/Br19ht/VisionScorer/main/nba_vid.mp4

In [ ]:
# Object Detection on the video using b_best.pt model.
# This 28 seconds video contains 1618 frames(=images).

!yolo task=detect mode=predict model=/content/b_best.pt conf=0.5 source = nba_vid.mp4 data=/Bigger-Dataset-2/data.yaml

In [ ]:
# Object Detection on the video using s_best.pt model.
# This 28 seconds video contains 1618 frames(=images).

!yolo task=detect mode=predict model=/content/s_best.pt conf=0.5 source = nba_vid.mp4 data=/Basketball-Players-1/data.yaml

In [ ]:
# Converting the video from avi to mp4 and showing the "object detected" version of the video,
# Result of b_best.pt model.

path="/content/runs/detect/predict3/nba_vid.avi"

clip=VideoFileClip(path)
clip.ipython_display(width=860)

In [ ]:
# Converting the video from avi to mp4 and showing the "object detected" version of the video,
# Result of s_best.pt model.

path="/content/runs/detect/predict4/nba_vid.avi"

clip=VideoFileClip(path)
clip.ipython_display(width=860)